# 上下文管理器和With语句
- manage resources(资源管理)


**优秀博客推荐**
- [Python with Context Managers](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/)

以打开文件为例，理解上下文管理器的必要性。系统中的文件描述符是有限的，在实际的编程中，被打开的文件描述符需要及时到进行close来释放相关的资源，否则会导致资源耗尽从而出现问题。

In [9]:
files = []
for x in range(100000):
    files.append(open('foo.txt', 'w'))

OSError: [Errno 24] Too many open files: 'foo.txt'

## with语句

with语句适用于于对资源进行访问的场合。确保使用过程中不管是否发生异常，都会执行必要的“清理”操作，并释放资源。比如文件使用后自动关闭，线程中锁的自动获取和释放。

In [2]:
# 文件不存在时的情况
try:
    with open('./data.txt') as f:
        for line in f:
            print(line, end='')
except IOError as err:
    print("Read failed, ", err)

Read failed,  [Errno 2] No such file or directory: './data.txt'


## 上下文管理器

[segmentfault:上下文管理器](https://segmentfault.com/a/1190000018846260)

### 创建上下文管理器

#### `__enter__()`和`__exit__()`实现创建上下文管理器

最简单的方式是在class中添加两个方法：
- `__enter__()` 返回被管理的资源，例如文件描述符
- `__exit__()`进行资源清理工作，无返回值。

In [12]:
'''
创建上下文管理器
'''
class File():
    def __init__(self, fileName, mode):
        self._fileName = fileName
        self._mode = mode
    def __enter__(self):
        self.file = open(self._fileName, self._mode)
        return self.file
    def __exit__(self, *args):
        self.file.close()

'''
测试
    同样地打开多个文件，并没有显式地调用close()
    可是，通过上线文管理器，并没有出现问题。
'''
files = []
for i in range(10000):
    with File('foo.txt', 'w') as f:
        f.write("foo")
        files.append(f)

#### 通过contextlib来创建上下文管理器

其中，yield相当于__enter__(),yield之后的语句相当于__exit__()语句。

In [1]:
from contextlib import contextmanager

'''
创建上下文管理器
'''
@contextmanager
def openFile(fileName, mode= "r"):
    file = open(fileName, mode)
    yield file
    file.close()

'''
测试
    打开10000个文件，非显式代用close(),查看是否会成功。
'''
files = []
for i in range(10000):
    with openFile('foo.txt', 'w') as f:
        f.write("foo")
        files.append(f)

for file in files:
    if not file.closed:
        print("File's not closed")

通过继承contextlib.ContextDecorator创建上下文管理器

In [24]:
from contextlib import ContextDecorator

class makeParagraph(ContextDecorator):
    def __enter__(self):
        print("<p>")
        return self
    
    def __exit__(self, *argc):
        print("<p>")

@makeParagraph()
def createHtml(word):
    print(word)

createHtml("This is a paragraph.")

<p>
This is a paragraph.
<p>


#### functools实现上下文管理器

- total_ordering可以方便实现比较大小,此时需要实现`__lt__、__le__、__gt__、__ge__`的至少一个。

In [10]:
from functools import total_ordering

@total_ordering
class Pig:
    def __init__(self, price = 68, year = 2019):
        self.price = price
        self.year = year
   
    def __lt__(self, other):
        # 如果需要比较大小，必须实现<, >,  <=,  >=一个其中的一个
        print("Comparing two pig")
        return self.price < other.price

pig2019 = Pig()
pig2018 = Pig(18, 2018)
print(pig2019 < pig2018)
print("*"*40)
print(pig2019 > pig2018)

Comparing two pig
False
****************************************
Comparing two pig
True
******************** 1 ********************
False


- **partial将原函数的部分参数初始换为固定值，实际的调用中只需要传递其他未固定的其他参数即可。**

In [14]:
import functools

def add(a, b):
    return a+b
add = functools.partial(add, 6)
add(2)

8

- **wraps函数可以实现保留原函数属性的迭代器**

[知乎对装饰器的深度解析](https://zhuanlan.zhihu.com/p/45458873)

In [27]:
import time
from functools import wraps

def log(param):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kw):
            curTime = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
            print("{0} Call {1} with {2}".format(curTime, func.__name__, param), end="")
            return func(*args, **kw)
        return wrapper
    return decorator

@log("decorator with parameter ")
def add(a, b):
    return a +b

print(", result = ", add(2, 6))

2019-12-16 14:36:25 Call add with decorator with parameter , result =  8


#### 高阶函数实现迭代器
该方式实现的迭代器将会丢失原函数的属性，如果想要保留原函数的属性，可以考虑使用functools.wraps。

In [17]:
import time

def format_log(func):
    def handler(*args, **kw):
        curTime = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
        print("{0} Call {1} ".format(curTime, func.__name__), end = "")
        return func(*args, **kw)
    return handler

@format_log
def add(a, b):
    return a + b

print(add(2, 6))

2019-12-16 14:14:30 Call add 8


### 可用的上下文管理器

- threading的Lock objects
- zipfile.ZipFiles
- subprocess.Popen
- tarfile.TarFile
- telnetlib.Telnet
- pathlib.Path

In [26]:
from threading import Lock
lock = Lock()

def do_something_dangerous():
    with lock:
        raise Exception('oops I forgot this code could raise exceptions')

try:
    do_something_dangerous()
except:
    print('Got an exception')
#lock.acquire()
print('Got here')

Got an exception
Got here
